# Section 4.4: Time series

In [1]:
import csv
import torch
import numpy as np

In [2]:
bikes_numpy = np.loadtxt("./hour-fixed.csv",
                        dtype = np.float32,
                        delimiter = ',',
                        skiprows = 1,
                        converters = {1: lambda x: float(x[8:10])})
bikes = torch.from_numpy(bikes_numpy)
print(bikes)
print(bikes.shape)
print(bikes.stride())

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])
torch.Size([17520, 17])
(17, 1)


In [3]:
col_list = next(csv.reader(open("./hour-fixed.csv"), delimiter = ','))
print(col_list)

['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt']


In [4]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
print(daily_bikes.shape)
print(daily_bikes.stride())

torch.Size([730, 24, 17])
(408, 17, 1)


In [5]:
daily_bikes = daily_bikes.transpose(1, 2)
print(daily_bikes.shape)
print(daily_bikes.stride())

torch.Size([730, 17, 24])
(408, 1, 17)


In [6]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
print(first_day[:, 9])

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])


In [7]:
weather_onehot.scatter_(dim = 1, index = first_day[:,9].unsqueeze(1).long() - 1, value = 1.0)
print(weather_onehot)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])


In [8]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [9]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
print(daily_weather_onehot.shape)

torch.Size([730, 4, 24])


In [10]:
daily_weather_onehot.scatter_(dim = 1, index = daily_bikes[:,9,:].long().unsqueeze(1) - 1, value = 1.0)
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), 1)
print(daily_bikes.shape)

torch.Size([730, 21, 24])


In [11]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0)/3.0
temp = daily_bikes[:, 10, :]
max_temp = torch.max(temp)
min_temp = torch.min(temp)
daily_bikes[:, 10, :] = (temp - min_temp)/(max_temp - min_temp)
# or we can subtract mean temp and divide it to the standard deviation
daily_bikes[:, 10, :] = (temp - torch.mean(temp))/torch.std(temp)